# ReverseNumberFlow example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils
from aiflows.workers import run_dispatch_worker_thread

In [3]:
FLOW_MODULES_PATH = "./"

# Connect to CoLink server

To connect to a real server, run below cell instead of the one after.

In [4]:
#USER_JWT_PATH = "/home/staverm/workspace/coflows-dev/coflows/jwts.txt"
#user_jwt = open(USER_JWT_PATH).readline().strip()
#cl = CoLink("http://127.0.0.1:2021", user_jwt)

In [5]:
cl = coflows_utils.start_colink_server()

# Load flow config

Every flow instance gets initialized with a flow config specified in a yaml file.

The ``_target_`` key specifies the method name used to instantiate the flow. 

## Load config for ReverseNumberAtomicFlow

In [6]:
reverse_number_atomic_default_config = read_yaml_file(
    os.path.join(FLOW_MODULES_PATH, "ReverseNumberFlowModule/ReverseNumberAtomicFlow.yaml"))
print(json.dumps(reverse_number_atomic_default_config, indent=4))

{
    "name": "ReverseNumber",
    "description": "A flow that takes in a number and reverses it.",
    "_target_": "ReverseNumberFlowModule.ReverseNumberAtomicFlow.instantiate_from_default_config",
    "input_interface": {
        "_target_": "aiflows.interfaces.KeyInterface",
        "keys_to_select": [
            "number"
        ]
    },
    "output_interface": {
        "_target_": "aiflows.interfaces.KeyInterface",
        "keys_to_rename": {
            "output_number": "reversed_number"
        }
    }
}


## Load config for ReverseNumberSequentialFlow

In [7]:
reverse_number_sequential_default_config = read_yaml_file(
    os.path.join(FLOW_MODULES_PATH, "ReverseNumberFlowModule/ReverseNumberSequentialFlow.yaml"))
print(json.dumps(reverse_number_sequential_default_config, indent=4))

{
    "name": "ReverseNumberTwice",
    "description": "A sequential flow that reverses a number twice.",
    "_target_": "ReverseNumberFlowModule.ReverseNumberSequentialFlow.instantiate_from_default_config",
    "flow_type": "ReverseNumberSequentialFlow_served",
    "input_interface": [
        "number"
    ],
    "output_interface": [
        "output_number"
    ],
    "subflows_config": {
        "first_reverse_flow": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_type": "ReverseNumberAtomicFlow_served",
            "name": "ReverseNumberFirst",
            "description": "A flow that takes in a number and reverses it."
        },
        "second_reverse_flow": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_type": "ReverseNumberAtomicFlow_served",
            "name": "ReverseNumberSecond",
        

# Serve Flows

In [8]:
serve_utils.serve_flow(
    cl=cl,
    flow_type="ReverseNumberAtomicFlow_served",
    default_config=reverse_number_atomic_default_config,
    default_state=None,
    default_dispatch_point="coflows_dispatch",
)
serve_utils.serve_flow(
    cl=cl,
    flow_type="ReverseNumberSequentialFlow_served",
    default_config=reverse_number_sequential_default_config,
    default_state=None,
    default_dispatch_point="coflows_dispatch",
    # serve_mode="statefull" "stateless" "singleton"
)

Started serving at flows:ReverseNumberAtomicFlow_served.
Started serving at flows:ReverseNumberSequentialFlow_served.


True

# Start dispatch workers

In [9]:
run_dispatch_worker_thread(cl, dispatch_point="coflows_dispatch", flow_modules_base_path=FLOW_MODULES_PATH)

Dispatch worker started in attached thread.

~~~ Dispatch task ~~~
flow_type: ReverseNumberSequentialFlow_served
flow_id: 750745d8-6df4-45b5-88c9-e8276fc56bf5
message_ids: ['4e2fd72f-e5fe-407e-b76c-dcd543c6395c']

Called ReverseNumberSequential



In [10]:
run_dispatch_worker_thread(cl, dispatch_point="coflows_dispatch", flow_modules_base_path=FLOW_MODULES_PATH)

Dispatch worker started in attached thread.

~~~ Dispatch task ~~~
flow_type: ReverseNumberAtomicFlow_served
flow_id: fd50460f-d6b5-403c-b454-5a16e608d055
message_ids: ['d70bd97e-ecbe-4edf-9f83-56e2b54be205']

Called ReverseNumberAtomic


~~~ Dispatch task ~~~
flow_type: ReverseNumberAtomicFlow_served
flow_id: 8f0411b8-eaed-4d11-9b80-fb90d012a2df
message_ids: ['b24b0de0-0340-4603-a537-4d8fac28c9ea']

Called ReverseNumberAtomic



In [12]:
flow = serve_utils.recursive_mount(
    cl=cl,
    client_id="local",
    flow_type="ReverseNumberSequentialFlow_served",
    config_overrides=None,
    initial_state=None,
    dispatch_point_override=None,
)

Mounted fd50460f-d6b5-403c-b454-5a16e608d055 at flows:ReverseNumberAtomicFlow_served:mounts:local:fd50460f-d6b5-403c-b454-5a16e608d055
Mounted 8f0411b8-eaed-4d11-9b80-fb90d012a2df at flows:ReverseNumberAtomicFlow_served:mounts:local:8f0411b8-eaed-4d11-9b80-fb90d012a2df
Mounted 750745d8-6df4-45b5-88c9-e8276fc56bf5 at flows:ReverseNumberSequentialFlow_served:mounts:local:750745d8-6df4-45b5-88c9-e8276fc56bf5


In [13]:
print("Pushing to...\n", flow)
  
input_data = {"id": 0, "number": 81}
    
input_message = FlowMessage(
    data=input_data,
    src_flow="Coflows team",
    dst_flow=flow,
    is_input_msg=True,
)

flow.ask(input_message).get_data()

Pushing to...
 Name: Proxy_ReverseNumberSequentialFlow_served
Class name: AtomicFlow
Type: AtomicFlow
Description: Proxy of ReverseNumberSequentialFlow_served
Input keys: no input keys
Output keys: no output keys



{'output_number': 81}